# Robotics Project: Inverse Kinematics (1)

### Objective

The goal is to compute the joint angles of the MARA robot from a desired end-effector position and orientation.

In [1]:
import math
import numpy as np

## Step 0: Specifications

Here we can change the robot parameters according to the robot used and the desired end-effector position and orientation.

### Robot parameters

<div>
    <img src="mara.png" width="250"/>
</div>

In [2]:
# For the MARA robot:
d1 = 0.09122+0.1637  # distance from the base to joint 2
a2 = 0.19998         # length of Link 2
a3 = 0.1493+0.1607   # distance from joint 3 to the wrist center
d6 = 0.1468+0.2449   # distance from the wrist center to the end-effector

print("d1 = " + str(round(d1,5)))
print("a2 = " + str(round(a2,5)))
print("a3 = " + str(round(a3,5)))
print("d6 = " + str(round(d6,5)))

d1 = 0.25492
a2 = 0.19998
a3 = 0.31
d6 = 0.3917


### Desired end-effector position and orientation

As a first verification, some easily visualized cases were used:

- The robot stands straight up (i.e. is aligned with the $z$-axis).
    - $x=0$ ; $y=0$ ; $z=d_1+d_6+a_2+a_3$


- The robot moves in the $x$-direction.
    - $x=a_2$ ; $y=0$ ; $z=d_1+d_6+a_3$
    - $x=a_3$ ; $y=0$ ; $z=d_1+d_6+a_2$
    - $x=a_2+a_3$ ; $y=0$ ; $z=d_1+d_6$


- The robot moves in the $y$-direction.
    - $x=0$ ; $y=a_2$ ; $z=d_1+d_6+a_3$
    - $x=0$ ; $y=a_3$ ; $z=d_1+d_6+a_2$
    - $x=0$ ; $y=a_2+a_3$ ; $z=d_1+d_6$

In [3]:
# Desired end-effector position
pex = a3
pey = 0
pez = a2+d1+d6
# robot max. height = 1.1566

# Desired end-effector orientation
yaw = 0
pitch = 0  # angles in degrees
roll = 0

print("Desired position:")
print("pex = " + str(pex))
print("pey = " + str(pey))
print("pez = " + str(pez))

print("")

print("Desired orientation:")
print("yaw = " + str(yaw) + "°")
print("pitch = " + str(pitch) + "°")
print("roll = " + str(roll) + "°")

Desired position:
pex = 0.31
pey = 0
pez = 0.8466

Desired orientation:
yaw = 0°
pitch = 0°
roll = 0°


***

## Step 1: Computation of rotation matrix 0R6

The rotation matrix 0R6 is computed directly from the desired yaw-pitch-roll angles.

In [4]:
def isOrthogonal(R):
    # Check if a rotation matrix R is orthogonal
    
    shouldBeIdentity = np.dot(R,np.transpose(R))  # since R*R'=I
    I = np.identity(3)
    n = np.linalg.norm(I - shouldBeIdentity)
    
    return n < 1e-6  # the difference (I-R*R') should be small


cy,sy = math.cos(math.radians(yaw)), math.sin(math.radians(yaw))
Rx = np.array([[1,  0,   0],
               [0, cy, -sy],
               [0, sy,  cy]])  # [Spong p.50]
print(isOrthogonal(Rx))  # since rotation matrix is always orthogonal

cp,sp = math.cos(math.radians(pitch)), math.sin(math.radians(pitch))
Ry = np.array([[ cp, 0, sp],
               [  0, 1,  0],
               [-sp, 0, cp]])  # [Spong p.50]
print(isOrthogonal(Ry))

cr,sr = math.cos(math.radians(roll)), math.sin(math.radians(roll))
Rz = np.array([[cr, -sr, 0],
               [sr,  cr, 0],
               [ 0,   0, 1]])  # [Spong p.50]
print(isOrthogonal(Rz))

# rotation matrix of the end-effector orientation
R60 = np.dot(Rz,np.dot(Ry,Rx))
print(isOrthogonal(R60))

# Verification
R60_verif = np.array([[cr*cp, cr*sp*sy-sr*cy, cr*sp*cy+sr*sy],
                      [sr*cp, sr*sp*sy+cr*cy, sr*sp*cy-cr*sy],
                      [  -sp,          cp*sy,          cp*cy]])  # [Spong p.50]
print(R60.round(6) == R60_verif.round(6))

True
True
True
True
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


## Step 2: Computation of wrist center position

The wrist center position is computed from the desired end-effector position and the rotation matrix deduced from its orientation.

In [5]:
# wrist position
pwx = pex - d6 * R60[0,2]
pwy = pey - d6 * R60[1,2]
pwz = pez - d6 * R60[2,2]  # [Spong p.88]

print("pwx = " + str(pwx))
print("pwy = " + str(pwy))
print("pwz = " + str(pwz))

pwx = 0.31
pwy = 0.0
pwz = 0.45489999999999997


## Step 3: Inverse Position problem

The objective is to determine the arm joint angles ($\theta_1$, $\theta_2$, $\theta_3$). The equations used are provided in the slides.

### Computation of THETA1

In [6]:
theta1 = math.atan2(pwy,pwx)  # [Spong p.91]
print("theta1 = " + str(theta1*180/math.pi) + "°")

theta1 = 0.0°


### Computation of THETA3

In [7]:
r = math.sqrt(pwx**2+pwy**2)
s = pwz-d1
D = (r**2+s**2-a2**2-a3**2)/(2*a2*a3)
theta3 = math.atan2(math.sqrt(1-D**2),D)  # [Spong p.94]
print("theta3 = " + str(theta3*180/math.pi) + "°")

theta3 = 90.00000000000001°


### Computation of THETA2

In [8]:
c3,s3 = math.cos(theta3), math.sin(theta3)
theta2 = math.atan2(s,r) - math.atan2(a3*s3,a2+a3*c3)  # [Spong p.94]
print("theta2 = " + str(theta2*180/math.pi) + "°")

theta2 = -24.348136759179656°


## Step 4: Computation of rotation matrix 0R3

The rotation matrix 0R3 is computed from the obtained arm joint angles ($\theta_1$, $\theta_2$, $\theta_3$).

In [9]:
def pose(a, alpha, d, theta):
    
    r11 = math.cos(theta)
    r21 = math.sin(theta)
    r31 = 0
    
    r12 = - math.sin(theta) * math.cos(alpha)
    r22 = math.cos(theta) * math.cos(alpha)
    r32 = math.sin(alpha)
    
    r13 = math.sin(theta) * math.sin(alpha)
    r23 = - math.cos(theta) * math.sin(alpha)
    r33 = math.cos(alpha)
    
    px = a * math.cos(theta)
    py = a * math.sin(theta)
    pz = d
    
    T = np.array([[r11, r12, r13, px],
                  [r21, r22, r23, py],
                  [r31, r32, r33, pz],
                  [  0,   0,   0,  1]])  # [Spong p.69]
    
    return T


A1 = pose(0, math.pi/2, d1, theta1)
A2 = pose(a2, 0, 0, theta2)
A3 = pose(a3, 0, 0, theta3)  # [Spong p.97]

T30 = np.dot(np.dot(A1,A2),A3)

# wrist center position
x = T30[0,3]
y = T30[1,3]
z = T30[2,3]

# rotation matrix of the anthropomorphic arm orientation
R30 = T30[0:3,0:3]
print(isOrthogonal(R30))

# Verification
c1,s1 = math.cos(theta1), math.sin(theta1)
c23,s23 = math.cos(theta2+theta3), math.sin(theta2+theta3)
R30_verif = np.array([[c1*c23, -c1*s23,  s1],
                      [s1*c23, -s1*s23, -c1],
                      [   s23,     c23,   0]])  # [Spong p.97]
print(R30.round(6) == R30_verif.round(6))

True
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


### First comparison: wrist position

In [10]:
print("Actual wrist position:")
print("x = " + str(x))
print("y = " + str(y))
print("z = " + str(z))

print("")

print("Desired wrist position:")
print("pwx = " + str(pwx))
print("pwy = " + str(pwy))
print("pwz = " + str(pwz))

Actual wrist position:
x = 0.30999999999999994
y = 1.2245243344674383e-17
z = 0.45489999999999997

Desired wrist position:
pwx = 0.31
pwy = 0.0
pwz = 0.45489999999999997


## Step 5: Computation of rotation matrix 3R6

The rotation matrix 3R6 is computed from the previously-computed rotation matrices 0R6 and 0R3.

In [11]:
# rotation matrix of the wrist with respect to the anthropomorphic arm
R63 = np.dot(np.transpose(R30),R60)  # since 0R6 = 0R3 * 3R6

print(isOrthogonal(R63))

[[r11,r12,r13],
 [r21,r22,r23],
 [r31,r32,r33]] = R63

True


## Step 6: Inverse Orientation problem

The objective is to determine the wrist joint angles ($\theta_4$, $\theta_5$, $\theta_6$). The equations used correspond to the expressions of the $ZYZ$-Euler angles provided in the reference book "Robot Modeling and Control".

### Computation of THETA5

In [12]:
#theta5 = math.atan2(math.sqrt(1-(s1*r13-c1*r23)**2),s1*r13-c1*r23)  # [Spong p.98]
theta5 = math.atan2(math.sqrt(1-r33**2),r33)  # [Spong p.48]
print("theta5 = " + str(theta5*180/math.pi) + "°")

theta5 = 90.0°


### Computation of THETA4

In [13]:
#theta4 = math.atan2(-c1*s23*r13-s1*s23*r23+c23*r33,c1*c23*r13+s1*c23*r23+s23*r33) + math.pi/2  # [Spong p.98]
theta4 = math.atan2(r23,r13)  # [Spong p.48]
print("theta4 = " + str(theta4*180/math.pi) + "°")

theta4 = 24.34813675917964°


### Computation of THETA6

In [14]:
#theta6 = math.atan2(s1*r12-c1*r22,-s1*r11+c1*r21) + math.pi/2  # [Spong p.98]
theta6 = math.atan2(r32,-r31)  # [Spong p.48]
print("theta6 = " + str(theta6*180/math.pi) + "°")

theta6 = -90.0°


# Summary: Results

The inputs and the results obtained are shown here.

In [15]:
print("INPUTS:")
print("x = " + str(pex))
print("y = " + str(pey))
print("z = " + str(pez))
print("yaw = " + str(yaw) + "°")
print("pitch = " + str(pitch) + "°")
print("roll = " + str(roll) + "°")

print("")

print("OUTPUTS:")  # Some experimental changes have been performed to adapt the model-based results to the simulation.
print("theta1 = " + str(round(  theta1*180/math.pi + 90 ,10)) + "°")
print("theta2 = " + str(round( -theta2*180/math.pi + 90 ,10)) + "°")
print("theta3 = " + str(round(  theta3*180/math.pi      ,10)) + "°")
print("theta4 = " + str(round(  theta5*180/math.pi - 90 ,10)) + "°")
print("theta5 = " + str(round(  theta4*180/math.pi      ,10)) + "°")
print("theta6 = " + str(round(  theta6*180/math.pi + 90 ,10)) + "°")

INPUTS:
x = 0.31
y = 0
z = 0.8466
yaw = 0°
pitch = 0°
roll = 0°

OUTPUTS:
theta1 = 90.0°
theta2 = 114.3481367592°
theta3 = 90.0°
theta4 = 0.0°
theta5 = 24.3481367592°
theta6 = 0.0°


***

# Forward Kinematics

### Objective

The goal is to compute the end-effector position and orientation from the desired robot joint angles.

In [16]:
A4 = pose(0, -math.pi/2, 0, theta4)
A5 = pose(0, math.pi/2, 0, theta5)
A6 = pose(0, 0, d6, theta6)  # [Spong p.79]

T60 = np.dot(np.dot(np.dot(T30,A4),A5),A6)

# end-effector position
x = T60[0,3]
y = T60[1,3]
z = T60[2,3]

# rotation matrix of the end-effector orientation
R = T60[0:3,0:3]
print(isOrthogonal(R))

[[r11,r12,r13],
 [r21,r22,r23],
 [r31,r32,r33]] = R

# rotation matrix to ZYX Euler angles (roll-pitch-yaw)
phi = math.atan2(r21,r11)
theta = math.atan2(-r31,math.sqrt(r32**2+r33**2))
psi = math.atan2(r32,r33)  # [Spong p.50]

True


### Final comparison: end-effector position and orientation

In [17]:
print("Actual end-effector position:")
print("x = " + str(x))
print("y = " + str(y))
print("z = " + str(z))

print("")

print("Desired end-effector position:")
print("pex = " + str(pex))
print("pey = " + str(pey))
print("pez = " + str(pez))

Actual end-effector position:
x = 0.30999999999999994
y = 1.2245243344674381e-17
z = 0.8466

Desired end-effector position:
pex = 0.31
pey = 0
pez = 0.8466


In [18]:
print("Correct orientation?")
print(R.round(6) == R60.round(6))

print("")

print("Actual end-effector orientation:")
print("psi = " + str(psi))
print("theta = " + str(theta))
print("phi = " + str(phi))

print("")

print("Desired end-effector orientation:")
print("yaw = " + str(yaw))
print("pitch = " + str(pitch))
print("roll = " + str(roll))

Correct orientation?
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]

Actual end-effector orientation:
psi = -3.5032177303741655e-34
theta = -5.721188726109833e-18
phi = 1.2246467991473532e-16

Desired end-effector orientation:
yaw = 0
pitch = 0
roll = 0
